In [31]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import locale
import os
import csv
import fastparquet
locale.setlocale(locale.LC_ALL,'pt_BR.UTF-8')
import numpy as np
from pathlib import Path
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

In [32]:
import gc
gc.collect()


10396

# Leitura e conversão de arquivos


In [33]:
path = r'sprint3\assets\data'
list_file = os.listdir(path)
save_path = r'sprint3\assets\parquet'

In [34]:
for i in list_file:
    if ('telemetria_' in i):
        try:
            df = pd.read_csv(os.path.join(path,i),sep=',', encoding='utf-8')
            print('\n',i)
            display(df.head(3))
            df.to_parquet(os.path.join(save_path,i).replace('csv','parquet'),compression='gzip')

        except:
            df = pd.read_csv(os.path.join(path,i),sep=',', encoding='iso-8859-1')
            print('\n',i)
            display(df.head(3))
            df.to_parquet(os.path.join(save_path,i).replace('csv','parquet'),compression='gzip')

    elif ('telemetria_' not in i) and (i.endswith('.csv')):
        try:
            df = pd.read_csv(os.path.join(path,i),sep=';', encoding='utf-8')
            print('\n',i)
            display(df.head(3))
            df.to_parquet(os.path.join(save_path,i).replace('csv','parquet'),compression='gzip')
        except:
            df = pd.read_csv(os.path.join(path,i),sep=';', encoding='iso-8859-1')
            print('\n',i)
            display(df.head(3))
            df.to_parquet(os.path.join(save_path,i).replace('csv','parquet'),compression='gzip')
    else:
        None



 clientes_desde.csv


,CLIENTE,CLIENTE_DESDE
0,TFDICB,2023-01-27
1,TFCU91,2021-01-26
2,TFDDYV,2022-07-18



 contratacoes_ultimos_12_meses.csv


,CD_CLIENTE,QTD_CONTRATACOES_12M,VLR_CONTRATACOES_12M
0,T07544,1,"2104,64437581956"
1,T01872,2,"0,00840771473583952"
2,T05174,3,"1777,9121734701"



 dados_clientes.csv


...

,clienteid,eventduration,moduloid,productlineid,referencedatestart,slotid,statuslicenca,tcloud,clienteprime
0,TFBKX200,21911.000,818,13,2025-03-21,818,Conectado,NaN,NaN
1,T7579400,74.542,5614,6,2025-03-21,4164,Desconectado,NaN,NaN
2,T5142500,0.033,212,2,2025-03-21,3001,Desconectado,NaN,NaN



 telemetria_9.csv


,clienteid,eventduration,moduloid,productlineid,referencedatestart,slotid,statuslicenca,tcloud,clienteprime
0,T0379900,0.000,34,2,2025-03-20,0,Desconectado,NaN,NaN
1,T1001900,0.000,9,2,2025-03-20,4134,NaN,NaN,NaN
2,T1096700,78.574,533,3,2025-03-20,4177,Desconectado,NaN,NaN



 tickets.csv


,CODIGO_ORGANIZACAO,NOME_GRUPO,TIPO_TICKET,STATUS_TICKET,DT_CRIACAO,DT_ATUALIZACAO,BK_TICKET,PRIORIDADE_TICKET
0,TFCPWG,Fábrica Consinco,task,closed,2024-11-12,2025-01-02,21787048,low
1,TB2434,PC - Financeiro,question,closed,2024-05-07,2024-05-16,20066662,normal
2,TFBYNF,TOTVS Chef Corporativo,question,closed,2024-03-19,2024-04-03,19662250,low


In [4]:
gc.collect()

0

## Início do modelo



In [ ]:
# -----------------------------
# 1) Diretórios básica
# -----------------------------
DATA_DIR = Path(r"sprint3\assets\parquet") 
OUT_DIR  = Path(r"sprint3\assets\output")
OUT_DIR.mkdir(parents=True, exist_ok=True)


# Lista de arquivos esperados (pelo nome que você enviou)
arquivos = {
    # centrais
    "dados_clientes": DATA_DIR / "dados_clientes.parquet",
    "clientes_desde": DATA_DIR / "clientes_desde.parquet",
    "contratacoes_12m": DATA_DIR / "contratacoes_ultimos_12_meses.parquet",
    "historico": DATA_DIR / "historico.parquet",
    "mrr": DATA_DIR / "mrr.parquet",
    "tickets": DATA_DIR / "tickets.parquet",
    # NPS
    "nps_relacional": DATA_DIR / "nps_relacional.parquet",
    "nps_tx_aquisicao": DATA_DIR / "nps_transacional_aquisicao.parquet",
    "nps_tx_implantacao": DATA_DIR / "nps_transacional_implantacao.parquet",
    "nps_tx_onboarding": DATA_DIR / "nps_transacional_onboarding.parquet",
    "nps_tx_produto": DATA_DIR / "nps_transacional_produto.parquet",
    "nps_tx_suporte": DATA_DIR / "nps_transacional_suporte.parquet",
    # telemetria 1..11
    **{f"telemetria_{i}": DATA_DIR / f"telemetria_{i}.parquet" for i in range(1, 12)}
}

def ler_parquet(p: Path):
    try:
        return pd.read_parquet(p)
    except Exception:
        return None

def normalizar_colunas(df: pd.DataFrame) -> pd.DataFrame:
    df.columns = [c.strip().replace(" ", "_").replace("-", "_").replace(".", "_").lower() for c in df.columns]
    return df

gc.collect()

0

In [6]:
# -------------------------------------------
# 2) Leitura e união por família de dataframe
# -------------------------------------------
tabelas = {}
for nome, caminho in arquivos.items():
    df = ler_parquet(caminho)
    if df is not None and len(df.columns) > 0:
        tabelas[nome] = normalizar_colunas(df)

# Une telemetria
telemetria = pd.concat([df for k, df in tabelas.items() if k.startswith("telemetria_")], ignore_index=True) \
             if any(k.startswith("telemetria_") for k in tabelas) else None

# Une NPS transacional
nps_trans = pd.concat([tabelas[k] for k in tabelas if k.startswith("nps_tx_")], ignore_index=True) \
           if any(k.startswith("nps_tx_") for k in tabelas) else None

# NPS relacional
nps_rel = tabelas.get("nps_relacional")

# Outras bases
dados_clientes   = tabelas.get("dados_clientes")
clientes_desde   = tabelas.get("clientes_desde")
contratacoes_12m = tabelas.get("contratacoes_12m")
historico        = tabelas.get("historico")
mrr              = tabelas.get("mrr")
tickets          = tabelas.get("tickets")

gc.collect()

0

In [7]:
# -----------------------------
# 3) Achando colunas-chave (simples)
# -----------------------------
# Tentamos nomes comuns para CID, DATA, VALOR — se não achar, usamos casamento simples de nome de colunas
def coluna_cliente(df):
    if df is None: return None
    for c in ["cod_cliente", "cd_cliente", "metadata_codcliente", "cid", "cliente_id", "id_cliente"]:
        if c in df.columns: return c
    # primeira coluna que contenha "cliente" ou termine com "cid"
    for c in df.columns:
        if "cliente" in c or c.endswith("cid"):
            return c
    return None

def coluna_data(df):
    if df is None: return None
    for c in ["purchase_date","respondedat","responded_at","data","dt_evento","created_at","dt","mes","competencia"]:
        if c in df.columns: return c
    for c in df.columns:
        if "date" in c or "data" in c or c.startswith("dt"):
            return c
    return None

def coluna_valor(df):
    if df is None: return None
    for c in ["net_amount","valor","vlr","mrr","valor_total","amount","gross_amount"]:
        if c in df.columns: return c
    for c in df.columns:
        if ("valor" in c) or ("amount" in c) or ("mrr" in c):
            return c
    return None

gc.collect()

0

In [19]:
# -----------------------------
# 4) Monta EVENTS (para recency/frequency)
# -----------------------------
def montar_events(lista_df):
    frames = []
    for nome, df in lista_df:
        if df is None: 
            continue
        cid = coluna_cliente(df)
        dt  = coluna_data(df)
        if cid and dt and cid in df.columns and dt in df.columns:
            tmp = df[[cid, dt]].copy()
            tmp.columns = ["cid", "event_dt"]
            tmp["cid"] = tmp["cid"].astype(str)
            tmp["event_dt"] = pd.to_datetime(tmp["event_dt"], errors="coerce")
            tmp = tmp.dropna(subset=["cid", "event_dt"])
            frames.append(tmp)
    if frames:
        return pd.concat(frames, ignore_index=True)
    else:
        return pd.DataFrame(columns=["cid", "event_dt"])

events = montar_events([
    ("nps_rel", nps_rel),
    ("nps_trans", nps_trans),
    ("historico", historico),
    ("mrr", mrr),
    ("telemetria", telemetria),
    ("tickets", tickets)
])

gc.collect()

0

In [20]:
# -----------------------------
# 5) Monta MONETARY (somas por cliente)
# -----------------------------
def montar_monetary(lista_df):
    frames = []
    for nome, df in lista_df:
        if df is None:
            continue
        cid = coluna_cliente(df)
        val = coluna_valor(df)
        if cid and val and cid in df.columns and val in df.columns:
            tmp = df[[cid, val]].copy()
            tmp.columns = ["cid", "amount"]
            tmp["cid"]   = tmp["cid"].astype(str)
            tmp["amount"] = pd.to_numeric(tmp["amount"], errors="coerce")
            frames.append(tmp)
    if frames:
        out = pd.concat(frames, ignore_index=True)
        out["amount"] = out["amount"].fillna(0.0)
        return out
    else:
        return pd.DataFrame(columns=["cid", "amount"])

monetary = montar_monetary([
    ("mrr", mrr),
    ("contratacoes_12m", contratacoes_12m),
    ("historico", historico)
])

gc.collect()

0

In [21]:
# -----------------------------
# 6) Calcula RFM (Recency, Frequency, Monetary):
# -----------------------------
# Identifica comportamento, último evento, frequência e valores
if len(events) > 0:
    ref_date = events["event_dt"].max() + pd.Timedelta(days=1)
    last_evt = events.groupby("cid", as_index=False)["event_dt"].max().rename(columns={"event_dt":"last_event"})
    freq     = events.groupby("cid", as_index=False).size().rename(columns={"size":"frequency"})
    rfm = last_evt.merge(freq, on="cid", how="outer")
else:
    # se não houver events, cria vazio
    ref_date = pd.Timestamp.today()
    rfm = pd.DataFrame(columns=["cid","last_event","frequency"])

if len(monetary) > 0:
    money = monetary.groupby("cid", as_index=False)["amount"].sum().rename(columns={"amount":"monetary"})
    rfm = rfm.merge(money, on="cid", how="left")
else:
    rfm["monetary"] = 0.0

rfm["last_event"] = pd.to_datetime(rfm.get("last_event"), errors="coerce")
rfm["recency"]    = (ref_date - rfm["last_event"]).dt.days
rfm["frequency"]  = pd.to_numeric(rfm.get("frequency"), errors="coerce").fillna(0).astype(int)
rfm["monetary"]   = pd.to_numeric(rfm.get("monetary"), errors="coerce").fillna(0.0)

# guarda uma cópia "crua"
rfm.to_csv(OUT_DIR / "rfm_raw.csv", index=False)

# Para modelar, precisa de recency válido
rfm_model = rfm.dropna(subset=["recency"]).copy()

gc.collect()

18

In [22]:
# -----------------------------
# 7) KMeans (k=3) – simples
# -----------------------------
if len(rfm_model) >= 3:
    # log1p para suavizar, alterado pelo Helder
    X = np.log1p(rfm_model[["recency","frequency","monetary"]].values)   
    X = StandardScaler().fit_transform(X)                               
    km = KMeans(n_clusters=3, random_state=42, n_init=10).fit(X)
    rfm_model["cluster"] = km.labels_

    # perfis por cluster 
    profiles = (rfm_model.groupby("cluster")
                .agg(recency_mean=("recency","mean"),
                     frequency_mean=("frequency","mean"),
                     monetary_mean=("monetary","mean"),
                     customers=("cid","nunique"))
                .reset_index()
                .sort_values(["recency_mean","frequency_mean","monetary_mean"],
                             ascending=[True, False, False]))

    # rótulos mais amigáveis (colunas semelhantes feitas pela Liora)
    ordem = profiles["cluster"].tolist()
    nomes = ["Best","Loyal","At Risk"]  # se k=3
    mapa = {cl: nome for cl, nome in zip(ordem, nomes)}
    rfm_model["segment"] = rfm_model["cluster"].map(mapa)
else:
    profiles = pd.DataFrame(columns=["cluster","recency_mean","frequency_mean","monetary_mean","customers"])
    rfm_model["cluster"] = np.nan
    rfm_model["segment"] = np.nan

# salva resultados
rfm_model.to_csv(OUT_DIR / "rfm_with_clusters.csv", index=False)
profiles.to_csv(OUT_DIR / "cluster_profiles.csv", index=False)

gc.collect()

0

In [ ]:
# --------------------------------------
# 8) Tendência mensal de eventos + flag de alerta
# --------------------------------------
if len(events) > 0:
    # events = events[events["event_dt"].dt.year == 2024] #observar pq não é o ideal, ver de transformar de outra forma
    monthly = (events.set_index("event_dt")
               .groupby(pd.Grouper(freq="MS"))
               .agg(events=("cid", "count"))
               .reset_index())
    monthly["rolling_mean"] = monthly["events"].rolling(6, min_periods=3).mean()
    monthly["rolling_std"]  = monthly["events"].rolling(6, min_periods=3).std()
    monthly["lower_band"]   = monthly["rolling_mean"] - 2*monthly["rolling_std"]
    monthly["anomaly_low"]  = monthly["events"] < monthly["lower_band"]
else:
    monthly = pd.DataFrame(columns=["event_dt","events","rolling_mean","rolling_std","lower_band","anomaly_low"])

monthly.to_csv(OUT_DIR / "monthly_events_trend.csv", index=False)

gc.collect()

0

In [25]:
events

,cid,event_dt
0,TEZHBA,2023-04-28 16:45:59.763
1,T04819,2025-03-24 09:31:00.000
2,T11082,2024-11-22 15:42:20.000
3,T10872,2025-01-13 16:48:09.000
4,T01690,2025-02-17 11:01:30.000
...,...,...
30819175,TDC0GU00,2025-03-18 00:00:00.000
30819176,TDC0GU00,2025-03-18 00:00:00.000
30819177,T4645700,2025-03-18 00:00:00.000
30819178,T4308700,2025-03-18 00:00:00.000


In [29]:
monthly.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 75 entries, 0 to 74
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   event_dt      75 non-null     datetime64[ns]
 1   events        75 non-null     int64         
 2   rolling_mean  73 non-null     float64       
 3   rolling_std   73 non-null     float64       
 4   lower_band    73 non-null     float64       
 5   anomaly_low   75 non-null     bool          
dtypes: bool(1), datetime64[ns](1), float64(3), int64(1)
memory usage: 3.1 KB


In [ ]:
# -----------
# 9) Gráficos 
# -----------
# 9.1 média monetária por cluster
if len(profiles) > 0:
    plt.figure(figsize=(8,4))
    plt.bar(profiles["cluster"].astype(str), profiles["monetary_mean"])
    plt.title("Cluster monetary mean")
    plt.xlabel("Cluster"); plt.ylabel("Monetary (mean)")
    plt.tight_layout()
    plt.savefig(OUT_DIR / "chart_cluster_monetary_mean.png", dpi=150)
    plt.close()

# 9.2 dispersão Frequency x Monetary
if "cluster" in rfm_model.columns and rfm_model["cluster"].notna().any():
    plt.figure(figsize=(7,5))
    plt.scatter(np.log1p(rfm_model["frequency"]), np.log1p(rfm_model["monetary"]))
    plt.title("Log(Frequency) vs Log(Monetary)")
    plt.xlabel("log(Frequency)")
    plt.ylabel("log(Monetary)")
    plt.tight_layout()
    plt.savefig(OUT_DIR / "chart_scatter_freq_monetary.png", dpi=150)
    plt.close()

# 9.3 tendência mensal de eventos
if len(monthly) > 0:
    plt.figure(figsize=(10,4))
    plt.plot(monthly["event_dt"], monthly["events"])
    plt.title("Monthly Events Trend")
    plt.xlabel("Month"); 
    plt.ylabel("Events")
    plt.tight_layout()
    plt.savefig(OUT_DIR / "chart_monthly_events_trend.png", dpi=150)
    plt.close()

In [ ]:
# -----------------------------
# 10) Resumo executivo (pequeno)
# -----------------------------
exec_summary = pd.DataFrame({
    "Metric": [
        "Customers (RFM table)",
        "Customers modeled",
        "Total monetary (sum)",
        "Best segment size",
        "Months with low anomaly"
    ],
    "Value": [
        rfm["cid"].nunique() if "cid" in rfm.columns else 0,
        rfm_model["cid"].nunique() if "cid" in rfm_model.columns else 0,
        float(np.nansum(rfm_model["monetary"])) if "monetary" in rfm_model.columns else 0.0,
        int((rfm_model["segment"] == "Best").sum()) if "segment" in rfm_model.columns else 0,
        int(monthly["anomaly_low"].sum()) if "anomaly_low" in monthly.columns else 0
    ]
})
exec_summary.to_csv(OUT_DIR / "executive_summary.csv", index=False)

print("Artefatos gerados em:", OUT_DIR.resolve())
gc.collect()

Artefatos gerados em: C:\Users\helde\OneDrive\FIAP\ANO_2\FASE_05\Enterprise\sprint3\assets\output


9062